<a href="https://colab.research.google.com/github/Benji8bit/NetoNUMPY_53/blob/main/neto_numpy_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:


*   если источник traffic_source равен Yandex или Google, то в source_type ставится
organic;
*   для источников paid и email из России ставим ad;
*   для источников paid и email не из России ставим other;
*   все остальные варианты берём из traffic_source без изменений.









In [20]:
import pandas as pd

log = pd.read_csv('data/visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [21]:
log.loc[log.traffic_source.str.match('yandex|google'), 'source_type'] = 'organic'
log.loc[log.traffic_source.str.match('paid|email') & log.region.str.match('Russia'), 'source_type'] = 'ad'
log.loc[log.traffic_source.str.match('paid|email') & ~log.region.str.match('Russia'), 'source_type'] = 'other'
log.source_type.fillna(log.traffic_source, inplace=True)

In [22]:
log.groupby(['source_type', 'traffic_source', 'region']).count()[['visit_id']]

visit_id
source_type traffic_source region             
ad          email          Russia         2209
            paid           Russia         1581
direct      direct         Australia       222
                           Belarus         998
                           China           397
                           India           134
                           Russia         4052
                           Ukraine        1210
organic     google         Australia        91
                           Belarus         376
                           China           151
                           India            55
                           Russia         1435
                           Ukraine         399
            yandex         Australia        88
                           Belarus         446
                           China           171
                           India            59
                           Russia         1636
                           Ukraine         500
other       email          Australia       115
                           Belarus         569
                           China           252
                           India            82
                           Ukraine         648
            paid           Australia        74
                           Belarus         368
                           China           138
                           India            39
                           Ukraine         443

# Задание 2
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:
1.   Прочитайте содержимое файла с датафрейм.
2.   Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.





In [24]:
import pandas as pd

urls = pd.read_csv('data/URLs.txt')

In [25]:
urls[urls.url.str.contains('\/[0-9]{8}\-')].reset_index(drop=True).head()

,url
0,/politics/36188461-s-marta-zhizn-rossiyan-susc...
1,/world/36007585-tramp-pridumal-kak-reshit-ukra...
2,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
3,/video/36001498-poyavilis-pervye-podrobnosti-g...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [9]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [10]:
aggregated = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': ['min', 'max']})
aggregated['lifetime'] = aggregated.timestamp['max'] - aggregated.timestamp['min']
aggregated.head()

rating   timestamp             lifetime
        count         min         max         
userId                                        
1          20  1260759108  1260759205       97
2          76   835355395   835356246      851
3          51  1298861589  1298932787    71198
4         204   949778714   949982274   203560
5         100  1163373044  1163375145     2101

In [11]:
days = aggregated.loc[aggregated.rating['count'] > 100, 'lifetime'].mean() / (60 * 60 * 24)
print(f'Среднее время жизни пользователей выставивших более 100 оценок: {days} дней')

Среднее время жизни пользователей выставивших более 100 оценок: 463.8947621482917 дней


# Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
* аналогичную таблицу по типам выручки с указанием адреса клиента.

*Обратите внимание, что в процессе объединения таблиц данные не должны теряться.*

In [15]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Ленина 10', 'Пионерская 23', 'Левобережная 1а',
                    'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [16]:
revenues = pd.concat([rzd, auto, air]).groupby('client_id').sum().reset_index()
revenues

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0
